<a href="https://colab.research.google.com/github/NireAstro/1221-dark-matter/blob/main/Gravitational_Waves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as ac

$R_{bh} = \sqrt\frac{GM_⊙}{c^2}$

In [3]:
M_bh = 1 * u.solMass
R_bh = 2 * ac.G * M_bh / ac.c**2
print("The radius of a solar-mass black hole is ", R_bh.to(u.km))

The radius of a solar-mass black hole is  2.95325007610025 km


$M_{Total} = \frac{δtc^3}{16πG}$

In [4]:
delta_t = 0.01 * u.second
M_bh_total = delta_t * ac.c**3 / (16 * np.pi * ac.G)
print("Total mass of the merging black holes is", M_bh_total.to(u.solMass))

Total mass of the merging black holes is 40.390629275268466 solMass


In [6]:
h = 1e-21
delta_Earth = 1 * u.earthRad * h
print("Deformation of Earth is", delta_Earth.to(u.nm))

Deformation of Earth is 6.378099999999999e-06 nm


h = R_bh / distance, so distance = R_bh / h

In [7]:
R_bh = 2 * ac.G * 30 * u.solMass / (ac.c**2)
D = R_bh / h
print("Distance of the black hole merger is", D.to(1e6 * u.parsec))

Distance of the black hole merger is 2871.249504952705 1e+06 pc
